In [771]:
import numpy as np
import pandas as pd
import json

In [772]:
### Read in data:
tr_headers = ["index", "word", "pos_tag"]
train_df = pd.read_csv("./data/train", sep="\t", header=None)
train_df.columns = tr_headers

dev_df = pd.read_csv("./data/dev", sep="\t", header=None)
dev_df.columns = tr_headers

test_headers = ["index", "word"]
test_df = pd.read_csv("./data/test", sep="\t", header=None)
test_df.columns = test_headers

### 1) Vocabulary Creation:

In [773]:
train_df[train_df["word"].str.contains("2")]

,index,word,pos_tag
16,17,29,CD
356,11,28,CD
1049,19,352.7,CD
1143,11,8.12,CD
1162,12,8.22,CD
...,...,...,...
911847,25,2.125,CD
911854,32,46.125,CD
911976,26,32,CD
912056,7,"2,480",CD


In [774]:
# Slight cleaning on num:
train_df["word"] = train_df["word"].str.replace(r'^\d+|.\d+$', "<num>", regex=True)
dev_df["word"] = dev_df["word"].str.replace(r'^\d+|.\d+$', "<num>", regex=True)
test_df["word"] = test_df["word"].str.replace(r'^\d+|.\d+$', "<num>", regex=True)

In [775]:
# Get the count of each word:
#word-type = word
cnt_d = {}
for row in train_df.iterrows():
    if row[1]["word"] in cnt_d:
        cnt_d[row[1]["word"]] += 1
    else:
        cnt_d[row[1]["word"]] = 1

In [776]:
# Create unknown key:
threshold = 2 #No threshold = 1
unknown_cnt = 0
unknown_word_lst = []   #We want to keep track of unknown words but group together
for k, v in cnt_d.items():
    if v < threshold:
        unknown_cnt += v
        unknown_word_lst.append(k)
    else:
        continue
cnt_d["< unk >"] = unknown_cnt

In [777]:
# Sort the occurences in descending order:
cnt_d_sorted = {k: v for k, v in sorted(cnt_d.items(), key=lambda item: -item[1])}

In [779]:
### Write the vocab to vocab.txt
#punctuation and numbers also count as being part of vocabulary
i = 0
with open('vocab.txt', 'w') as f:
    f.write("< unk >")
    f.write("\t")
    f.write(str(i))
    f.write("\t")
    f.write(str(cnt_d_sorted["< unk >"]))
    f.write("\n")
    i+=1
    for k, v in cnt_d_sorted.items():
        if k == "< unk >":
            continue
        elif v >= threshold:
            f.write(k)
            f.write("\t")
            f.write(str(i))
            f.write("\t")
            f.write(str(v))
            f.write("\n")
            i+=1
        

In [780]:
print("Selected threshold:", threshold)

Selected threshold: 2


In [781]:
print("Total Size of Vocab:", i)

Total Size of Vocab: 21436


In [782]:
print("Total occurences of < unk >:", cnt_d_sorted["< unk >"])

Total occurences of < unk >: 17019


In [783]:
#Read in the vocab file:
vocab_d = {} #want just 0,word1... 90,word90 mapping
vocab_file = open("vocab.txt", "r").read().splitlines()


for line in vocab_file:
    line_split = line.split("\n")
    for actual_line in line_split:
        actual_line_split = actual_line.split("\t")
        if len(actual_line_split) == 1:
            break
        vocab_d[int(actual_line_split[1])] = actual_line_split[0]

### 2. Model Learning

In [786]:
train_df.head()

,index,word,pos_tag
0,1,Pierre,NNP
1,2,Vinken,NNP
2,3,",",","
3,4,<num>,CD
4,5,years,NNS


In [787]:
### Transition Prob. must create prob of all pos_tag transitions, contain in dictionary:
pos_tag_d = {}
all_pos_tags = train_df["pos_tag"].values
for i, pos_tag in enumerate(all_pos_tags):
    if i == (len(all_pos_tags) - 1):
        break
    
    next_tag = all_pos_tags[i+1]
    if pos_tag in pos_tag_d:
        if next_tag in pos_tag_d[pos_tag]:
            pos_tag_d[pos_tag][next_tag] += 1
        else:
            pos_tag_d[pos_tag][next_tag] = 1
            
    else:
        pos_tag_d[pos_tag] = {}
        pos_tag_d[pos_tag][next_tag] = 1

In [788]:
# Need count of transition state individually:
pos_total_counts_d = {}
for k, v in pos_tag_d.items():
    pos_total_counts_d[k] = 0
    for k_inner, v_inner in v.items():
        pos_total_counts_d[k] += v_inner

In [789]:
# Create transtion prob:
transition_d = {}
for k, v in pos_tag_d.items():
    transition_d[k] = {}
    for k_inner, v_inner in v.items():
        transition_d[k][k_inner] = v_inner/pos_total_counts_d[k]

In [790]:
# Format transition d as wanted:
transition_d_formatted = {}
for k, v in transition_d.items():
    for k_inner, v_inner in v.items():
        key_str = str(k) + ", " + str(k_inner)
        transition_d_formatted[key_str] = transition_d[k][k_inner]

In [792]:
### Emission Prob - Must create prob of word given POS tag. Check if word is in Unknown Lst:
#Takes ~3 min to run
pos_to_word_d = {}
all_pos_tags = train_df["pos_tag"].values
all_words = train_df["word"].values
for i, pos_tag in enumerate(all_pos_tags):
    word = all_words[i]
    if pos_tag in pos_to_word_d:
        if word in unknown_word_lst:
            if "< unk >" in pos_to_word_d[pos_tag]:
                pos_to_word_d[pos_tag]["< unk >"] += 1
            else:
                pos_to_word_d[pos_tag]["< unk >"] = 1
        else:
            if word in pos_to_word_d[pos_tag]:
                pos_to_word_d[pos_tag][word] += 1
            else:
                pos_to_word_d[pos_tag][word] = 1
            
    else:
        pos_to_word_d[pos_tag] = {}
        pos_to_word_d[pos_tag][word] = 1

In [794]:
# Create emission prob:
emission_d = {}
for k, v in pos_to_word_d.items():
    emission_d[k] = {}
    for k_inner, v_inner in v.items():
        emission_d[k][k_inner] = v_inner/pos_total_counts_d[k]

In [795]:
# Format emission d as wanted:
emission_d_formatted = {}
for k, v in emission_d.items():
    for k_inner, v_inner in v.items():
        key_str = str(k) + ", " + str(k_inner)
        emission_d_formatted[key_str] = emission_d[k][k_inner]

In [796]:
# Consolidate emission/transition:
e_t_results_d = {}
e_t_results_d["transition"] = transition_d_formatted
e_t_results_d["emission"] = emission_d_formatted

In [797]:
# Write the Emission/Transition Prob to a file:
with open('hmm.json', 'w') as f:
    json.dump(e_t_results_d, f)

In [798]:
print("# of Transition Parameters:", len(transition_d_formatted))

# of Transition Parameters: 1378


In [799]:
print("# of Emission Parameters:", len(emission_d_formatted))

# of Emission Parameters: 28532


In [800]:
# Load in Emission/Transition json:
hmm = open("hmm.json", "r")
e_t_model = json.load(hmm)

### 3. Greedy Decoding with HMM:

In [801]:
# s1* = arg max t(s1)e(x1|s1)
# s2* = arg max t(s2|s1*)e(x2|s2)

In [802]:
# Calculate best odds to be t(s1):
most_likely_start_d = {}
pos_tags_start = train_df[train_df["index"] == 1]["pos_tag"]
for pos_tag in pos_tags_start:
    if pos_tag in most_likely_start_d:
        most_likely_start_d[pos_tag] += 1
    else:
        most_likely_start_d[pos_tag] = 1

In [803]:
most_likely_start_prob_d = {k:v/len(pos_tags_start) for k,v in most_likely_start_d.items()}

In [804]:
most_likely_start_prob_d

{'NNP': 0.19789104610393007,
 'DT': 0.21911141347009264,
 'IN': 0.1288398137003506,
 'PRP': 0.06148935056779528,
 'EX': 0.004238840337013972,
 '``': 0.07472918520069077,
 'CD': 0.011225077188759224,
 'RBR': 0.0020932544874143074,
 'NNS': 0.041237113402061855,
 'NN': 0.0411847820398765,
 'JJ': 0.041708095661730074,
 'JJR': 0.0017007692710241248,
 'RB': 0.05604688890051808,
 'WRB': 0.00609660369459417,
 'CC': 0.05691035637657648,
 'VBG': 0.012010047621539588,
 'WDT': 0.0008111361138730441,
 'VBN': 0.005834946883667382,
 '-LRB-': 0.003427704223140928,
 'VB': 0.0030613846878434245,
 'WP': 0.003113716050028782,
 'PRP$': 0.007797372965618295,
 'TO': 0.0035323669475116437,
 'JJS': 0.00248573970380449,
 'NNPS': 0.0020409231252289496,
 'VBZ': 0.001517609503375373,
 'VBD': 0.0007588047516876865,
 'LS': 0.0009157988382437595,
 "''": 0.0003663195352975038,
 ':': 0.002799727876916636,
 'VBP': 0.0003663195352975038,
 'PDT': 0.0007326390705950076,
 'UH': 0.0006279763462242922,
 'MD': 0.00054947930294

In [806]:
def greedy_hmm(df):
    """Allows for a greedy implementation of HMM. Input - Dataframe that has words as column."""
    pred_lst = []
    all_words = df["word"].values
    prev_pos = "None"
    for i, word in enumerate(all_words):
        best_prob = 0
        best_pos = "None"
        if (word in unknown_word_lst) or (word not in cnt_d_sorted):
            word = "< unk >"
        if i == 0:
            for pos_tag in emission_d.keys():
                if word in emission_d[pos_tag]: 
                    e_x1_s1 = emission_d[pos_tag][word]*most_likely_start_prob_d[pos_tag]
                    if e_x1_s1 > best_prob:
                        best_prob = e_x1_s1
                        best_pos = pos_tag
                else:
                    continue
            pred_lst.append(best_pos)
            prev_pos = best_pos
        else:
            for pos_tag in emission_d.keys():
                if word in emission_d[pos_tag]:
                    e_x2_s2 = emission_d[pos_tag][word]
                    if pos_tag in transition_d[prev_pos]:
                        t_s2_s1 = transition_d[prev_pos][pos_tag]
                    else:
                        t_s2_s1 = 0
                    prob_ = e_x2_s2*t_s2_s1
                    if prob_ > best_prob:
                        best_prob = prob_
                        best_pos = pos_tag
                else:
                    continue

            if best_pos == "None": #Edge case where word and POS DO NOT Allign
                for pos_tag in emission_d.keys():
                    if word in emission_d[pos_tag]:
                        e_x2_s2 = emission_d[pos_tag][word]
                        if e_x2_s2 > best_prob:
                            best_prob = e_x2_s2
                            best_pos = pos_tag
            pred_lst.append(best_pos)
            prev_pos = best_pos
            
    return pred_lst

In [807]:
# Predict on training data:
train_pred_lst = greedy_hmm(train_df)

In [808]:
training_acc = sum(np.array(train_pred_lst) == train_df["pos_tag"].values)/len(np.array(train_pred_lst))
training_acc

0.9507014071999079

In [809]:
# Predict on Dev Data:
dev_pred_lst = greedy_hmm(dev_df)

In [810]:
dev_acc = sum(np.array(dev_pred_lst) == dev_df["pos_tag"].values)/len(np.array(dev_pred_lst))
dev_acc

0.9377618238115476

In [811]:
# Predict on Test Data:
test_pred_lst = greedy_hmm(test_df)

In [812]:
# Write output of Test Data:
with open("greedy.out", "w") as g:
    test_idx = test_df["index"].values
    test_word = test_df["word"].values
    for i, pred in enumerate(test_pred_lst):
        g.write(str(test_idx[i]))
        g.write("\t")
        g.write(str(test_word[i]))
        g.write("\t")
        g.write(str(pred))
        g.write("\n")

### 4. Viterbi Decoding With HMM:

In [815]:
def viterbi_hmm(df):
    """Allows for a viterbi decoding implementation of HMM. Input - Dataframe that has words as column."""
    all_words = df["word"].values
    states = [k for k in emission_d.keys()]
    V = []
    for i, word in enumerate(all_words):
        V.append({})
        current_map = {}
        if (word in unknown_word_lst) or (word not in cnt_d_sorted):
            word = "< unk >"
        if i == 0:
            for pos_tag in states:
                if word in emission_d[pos_tag]: 
                    e_x1_s1 = np.log(emission_d[pos_tag][word]) + np.log(most_likely_start_prob_d[pos_tag])
                    current_map[pos_tag] = {"prob": e_x1_s1, "prev_state": None}
                else:
                    current_map[pos_tag] = {"prob": -50, "prev_state": None}
            V[i] = current_map
        else:
            for pos_tag in states:
                if pos_tag in transition_d[states[0]]:
                    best_prob = V[i-1][states[0]]["prob"] + np.log(transition_d[states[0]][pos_tag])
                else:
                    best_prob = -1000000
                past_st = states[0]
                for prev_tag in states[1:]:
                    if pos_tag in transition_d[prev_tag]:
                        t_s2_s1 = np.log(transition_d[prev_tag][pos_tag])
                    else:
                        t_s2_s1 = -100
                    tr_cost = V[i-1][prev_tag]["prob"] + t_s2_s1 #log or -100 penalty
                    if tr_cost > best_prob:
                        best_prob = tr_cost
                        past_st = prev_tag
                
                if word in emission_d[pos_tag]:
                    e_x1_s1 = np.log(emission_d[pos_tag][word])
                else:
                    e_x1_s1 = -100
                best_prob = best_prob + e_x1_s1  #log or -100 penalty
                V[i][pos_tag] = {"prob": best_prob, "prev_state": past_st}
                
    pred_lst = []
    final_state = "." #init a guess of . ending
    best_prob = -np.inf #init a guess of best prob
    
    #Figure out what the best final state is:
    for st, d_ in V[len(all_words)-1].items():
        if d_["prob"] > best_prob:
            final_state = st
            best_prob = d_["prob"]
            
    #Follow path from end to beginning      
    for i in range(len(all_words)-2, -1, -1):
        pred_lst.append(V[i+1][final_state]["prev_state"])
        final_state = V[i+1][final_state]["prev_state"]
        
    return np.flip(np.array(pred_lst),axis=0)

In [816]:
# Predict on Dev Data:
dev_pred_lst = viterbi_hmm(dev_df)

In [817]:
dev_acc = sum(1 for x,y in zip(dev_pred_lst, dev_df["pos_tag"].values) if x == y)/len(dev_pred_lst)
dev_acc

0.9505187186473093

In [818]:
# Predict on Test Data:
test_pred_lst = viterbi_hmm(test_df)

In [819]:
# Write output of Test Data:
with open("viterbi.out", "w") as v:
    test_idx = test_df["index"].values
    test_word = test_df["word"].values
    for i, pred in enumerate(test_pred_lst):
        v.write(str(test_idx[i]))
        v.write("\t")
        v.write(str(test_word[i]))
        v.write("\t")
        v.write(str(pred))
        v.write("\n")